# Preguntas de negocio en una tienda de música

En este proyecto se estudian algunas preguntas de negocio sobre una base de datos de una tienda de música ficticia llamada chinook.

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

In [2]:
%%sql
SELECT
    name, 
    type
FROM sqlite_master
WHERE type IN ('table', 'view')

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql
SELECT * FROM invoice
LIMIT 3

Done.


invoice_id,customer_id,invoice_date,billing_address,billing_city,billing_state,billing_country,billing_postal_code,total
1,18,2017-01-03 00:00:00,627 Broadway,New York,NY,USA,10012-2612,15.84
2,30,2017-01-03 00:00:00,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,9.9
3,40,2017-01-05 00:00:00,"8, Rue Hanovre",Paris,None,France,75002,1.98


In [4]:
%%sql
SELECT * FROM customer
LIMIT 3

Done.


customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,fax,email,support_rep_id
1,Luís,Gonçalves,Embraer - Empresa Brasileira de Aeronáutica S.A.,"Av. Brigadeiro Faria Lima, 2170",São José dos Campos,SP,Brazil,12227-000,+55 (12) 3923-5555,+55 (12) 3923-5566,luisg@embraer.com.br,3
2,Leonie,Köhler,None,Theodor-Heuss-Straße 34,Stuttgart,None,Germany,70174,+49 0711 2842222,None,leonekohler@surfeu.de,5
3,François,Tremblay,None,1498 rue Bélanger,Montréal,QC,Canada,H2G 1A7,+1 (514) 721-4711,None,ftremblay@gmail.com,3


## Géneros más vendidos en USA

La tienda considera la compra de álbumes de 4 artistas para vender su música a clientes en USA. Los artistas y sus géneros respectivos son los siguientes:

|Artista             |Genero |
|--------------------|-------|
|Regal               |Hip-Hop|
|Red Tone            |Punk   |
|Meteor and the Girls|Pop    |
|Slim Jim Bites      |Blues  |

La tienda no tiene a la venta ningún material de estos artistas y se quiere saber cuáles son los tres mejores artistas cuyos álbumes debería comprar la tienda con el fin de venderlos en USA.

In [5]:
%%sql

WITH usa_tracks AS
    (
     SELECT il.* FROM invoice_line il
     INNER JOIN invoice i ON i.invoice_id = il.invoice_id
     INNER JOIN customer c ON c.customer_id = i.customer_id
     WHERE c.country = 'USA'
    )

SELECT
    g.name genre,
    SUM(usa.quantity) tracks_sold,
    ROUND(100 * CAST(SUM(usa.quantity) AS Float) / (SELECT SUM(quantity) FROM usa_tracks), 2) percent
FROM usa_tracks usa
INNER JOIN track t ON t.track_id = usa.track_id
INNER JOIN genre g ON g.genre_id = t.genre_id
GROUP BY 1
ORDER BY 2 DESC
LIMIT 10

Done.


genre,tracks_sold,percent
Rock,561,53.38
Alternative & Punk,130,12.37
Metal,124,11.8
R&B/Soul,53,5.04
Blues,36,3.43
Alternative,35,3.33
Latin,22,2.09
Pop,22,2.09
Hip Hop/Rap,20,1.9
Jazz,14,1.33


De acuerdo a la tabla de ventas en Estados Unidos según el género, se puede ver que entre los géneros Hip-Hop, Punk, Pop y Blues de las bandas a promocionar, las tres con mejores generos son Punk, Blues y Pop en ese orden.

Por esto, los tres artistas recomendados para añadir a la tienda serían 
- Red Tone (Punk)
- Slim Jim Bites (Blues) 
- Meteor and the Girls (Pop).

## Desempeño de los vendedores

A cada cliente se le relaciona con el vendedor al cual hizo la compra. Se quiere estudiar el desempeño de los vendedores de la tienda considerando la cantidad total de dólares en venta que haya generado para determinar si hay vendedores con mejor o menor desempeño.

In [6]:
%%sql
SELECT
    e.first_name || ' ' || e.last_name name,
    e.hire_date,
    ROUND(SUM(i.total), 2) total_sales,
    COUNT(i.customer_id) sales,
    ROUND(SUM(i.total) / COUNT(i.customer_id), 2) avg_sale_price
FROM invoice i
LEFT JOIN customer c ON c.customer_id = i.customer_id
LEFT JOIN employee e ON e.employee_id = c.support_rep_id
GROUP BY e.employee_id
ORDER BY 3 DESC

Done.


name,hire_date,total_sales,sales,avg_sale_price
Jane Peacock,2017-04-01 00:00:00,1731.51,212,8.17
Margaret Park,2017-05-03 00:00:00,1584.0,214,7.4
Steve Johnson,2017-10-17 00:00:00,1393.92,188,7.41


In [7]:
%%sql

SELECT 
    i.invoice_date last_sale_date
FROM invoice i
ORDER BY i.invoice_id DESC
LIMIT 1

Done.


last_sale_date
2020-12-30 00:00:00


No parece haber una gran diferencia en el desempeño entre Steve y Margaret, pues aunque Steve ha vendido menos, mantiene una cantidad de precio promedio por venta similar a la de Margaret. Sin embargo, Jane parece tener una cuenta promedio de venta mayor a sus compañeros. Esto podría ser a razón de que Jane fue contratada antes (1 mes antes que Margaret y 6 meses antes que Steve).

## Estadísticas por paises

Se quiere estudiar las ventas segmentadas por país para identificar los paises que muestran potencial para incrementar las ventas.

Por cada país con más de un cliente, se quiere considerar las siguientes estadísticas:

- Total de ventas en USD
- Número de clientes
- Valor promedio de ventas por cliente

Los paises con únicamente 1 cliente son agregados al grupo 'Other' que aparece al final de la tabla

In [8]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT
    country,
    total_customers,
    total_sales,
    avg_order_value
FROM 
    (
     SELECT
         country,
         COUNT(DISTINCT customer_id) total_customers,
         ROUND(SUM(total), 2) total_sales,
         ROUND(SUM(total) / COUNT(invoice_id), 2) avg_order_value,
         CASE
             WHEN country = 'Other' THEN 1
             ELSE 0
         END AS sort
     FROM country_or_other 
     GROUP BY country
     ORDER BY sort, total_sales DESC
    );

Done.


country,total_customers,total_sales,avg_order_value
USA,13,1040.49,7.94
Canada,8,535.59,7.05
Brazil,5,427.68,7.01
France,5,389.07,7.78
Germany,4,334.62,8.16
Czech Republic,2,273.24,9.11
United Kingdom,3,245.52,8.77
Portugal,2,185.13,6.38
India,2,183.15,8.72
Other,15,1094.94,7.45


La anterior tabla muestra los resultados por país ordenados por las ventas totales. Los paises a considerar que pueden aportar un mayor ingreso por ventas son aquellos que tienen un alto valor de monto promedio de venta.

Los tres primeros paises según esta estadística son:

In [9]:
%%sql
WITH country_or_other AS
    (
     SELECT
       CASE
           WHEN (
                 SELECT count(*)
                 FROM customer
                 where country = c.country
                ) = 1 THEN "Other"
           ELSE c.country
       END AS country,
       c.customer_id,
       i.*
     FROM invoice i
     INNER JOIN customer c ON c.customer_id = i.customer_id
    )
    
SELECT
    country,
    total_customers,
    total_sales,
    avg_order_value
FROM 
    (
     SELECT
         country,
         COUNT(DISTINCT customer_id) total_customers,
         ROUND(SUM(total), 2) total_sales,
         ROUND(SUM(total) / COUNT(invoice_id), 2) avg_order_value,
         CASE
             WHEN country = 'Other' THEN 1
             ELSE 0
         END AS sort
     FROM country_or_other 
     GROUP BY country
     ORDER BY sort, avg_order_value DESC
     LIMIT 3
    );

Done.


country,total_customers,total_sales,avg_order_value
Czech Republic,2,273.24,9.11
United Kingdom,3,245.52,8.77
India,2,183.15,8.72


Se recomienda entonces aumentar la publicidad en estos paises, aunque dado el bajo número de clientes en estos paises es preferible hacerlo mediante campañas pequeñas con el fin de comprobar si estos datos se replican con los nuevos clientes.

## Albums vs Single tracks

Los clientes tienen dos opciones para comprar música en esta tienda: comprar un álbum completo o seleccionar singles. 

Se está considerando la opción de únicamente comprar los singles más populares de cada álbum con el fin de ahorrar costos. La pregunta a considerar es qué porcentaje de las ventas viene de ventas de albumes completos y qué porcentaje de venta de singles para entender el efecto de esta opción en los ingresos.

In [10]:
%%sql

WITH first_track AS
    ( 
     SELECT 
         il.invoice_id invoice_id,
         MIN(il.track_id) first_track_id
     FROM invoice_line il
     GROUP BY 1
    ),
    
track_album_purchase AS
    (
     SELECT
     ft.invoice_id,
     CASE 
         WHEN
         ( 
          SELECT t.track_id FROM track t
          WHERE t.album_id = (
                              SELECT t2.album_id FROM track t2
                              WHERE t2.track_id = ft.first_track_id             
                             )
         
          EXCEPT
            
          SELECT il2.track_id FROM invoice_line il2
          WHERE il2.invoice_id = ft.invoice_id
         ) IS NULL
        
         AND
        
         (
          SELECT il2.track_id FROM invoice_line il2
          WHERE il2.invoice_id = ft.invoice_id
            
          EXCEPT 
            
          SELECT t.track_id FROM track t
          WHERE t.album_id = (
                              SELECT t2.album_id FROM track t2
                              WHERE t2.track_id = ft.first_track_id             
                             )
         ) IS NULL
         THEN 'yes'
         ELSE 'no'
     END AS 'album_purchase'
     FROM first_track ft
    )
    
SELECT
    album_purchase,
    COUNT(invoice_id) number_of_invoices,
    ROUND(100 * CAST(COUNT(invoice_id) AS Float) / (SELECT COUNT(*) FROM invoice), 2) percentage
FROM track_album_purchase
GROUP BY 1


Done.


album_purchase,number_of_invoices,percentage
no,500,81.43
yes,114,18.57


Como se puede ver en la tabla anterior, el 18.5% de las ventas que se registran en la tienda corresponden a ventas de albumes completos de música. Por esto, no es recomendable eliminar la opción de vender albumes completos y pasar a vender únicamente singles, pues esto corresponde a una gran porción de las ventas e ingresos.